In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import spikeinterface as si
import matplotlib.pyplot as plt
import os
from matplotlib.backends.backend_pdf import PdfPages
import torch
import torch.nn.functional as F


In [2]:
select_neurons = ['Neuron_1', 'Neuron_3', "Neuron_5", "Neuron_6","Neuron_7", "Neuron_8",
                  'Neuron_10', 'Neuron_11', 'Neuron_12', "Neuron_14","Neuron_13",
                  'Neuron_16', "Neuron_18", "Neuron_20","Neuron_22", "Neuron_26", "Neuron_25", "Neuron_27", "Neuron_29","Neuron_28"]

spike_inf = pd.read_csv("/media/ubuntu/sda/data/mouse11/output/analysis_grating/02_merge_new/spike_inf_refinhe.tsv", sep = '\t')
cluster_inf = pd.read_csv("/media/ubuntu/sda/data/mouse11/output/analysis_grating/02_merge_new/clusteri_inf_refinhe.tsv", sep = '\t')

In [3]:
trigger_time = pd.read_csv("/media/ubuntu/sda/data/mouse11/output/analysis_grating/01_get_trigger/trigger_time_dynamic.tsv", sep = '\t').iloc[:, 1:]

import pickle
with open('/media/ubuntu/sda/data/mouse11/output/analysis_grating/02_merge_new/outlier_dynamic.pkl', 'rb') as f:
    outlier = pickle.load(f)


In [4]:
trigger_time['start'] = trigger_time['start'] + 10000
trigger_time['expand_end'] = trigger_time['start'] + 25000
trigger_time['end'] = trigger_time['start'] + 20000
trigger_time['expand_start'] = trigger_time['start'] - 5000

In [5]:
for i in outlier:
    date = i.split("_")[0]
    image = i.split("_")[1] + "_" + i.split("_")[2]
    trigger_time = trigger_time[~((trigger_time['date'] == int(date)) & (trigger_time['order'] == image))]

In [6]:
date_order = ['030122', '032322', '042322', '052322', '052422', '062422', '072322', '082322', '092222', '112822', '122322', '012123', '022423', '032323',
              '042323', '052423', '062323', '072123']

In [7]:
filter_outlier = {}
with PdfPages("raster_overall.pdf") as pdf:
    for neuron in spike_inf['Neuron'].unique():
        filter_outlier[neuron] = []
        neuron_df = spike_inf[spike_inf['Neuron'] == neuron]

        fig, ax = plt.subplots(figsize = (12, 7.5))
        for index, row in trigger_time.iterrows():
            date = row['date']
            image_order = row['order']
            start = row['expand_start'] 
            end = row['expand_end']
            
            filtered_spikes = neuron_df[(neuron_df['date'] == date) & 
                                        (neuron_df['time'] >= start) & 
                                        (neuron_df['time'] <= end)]
            
            if not filtered_spikes.empty:
                ax.plot(filtered_spikes['time'] - start, [index] * len(filtered_spikes), marker='|', mew=1, markersize=3, ls='', color='k')
            else:
                filter_outlier[neuron].append(index)

        ax.set_yticks([])
        #ax.axvspan(1 + (5000 - 1) * 0.5, 1 + (5000 - 1), color='gray', alpha=0.3)

        ax.set_xlabel('time (s)')
        ax.set_ylabel("")
        ax.set_title(f'Raster Plot for {neuron}')

        pdf.savefig(fig)
        plt.close()

In [8]:
from collections import Counter
counter = Counter()

for key in filter_outlier.keys():
    filtered_elements = [item for item in filter_outlier[key]]
    counter.update(filtered_elements)

filtered_counter = {element: count for element, count in counter.items() if count == 20}
filtered_counter = list(filtered_counter.keys())

trigger_time = trigger_time.drop(filtered_counter)
trigger_time = trigger_time.reset_index(drop = True)

In [9]:
filter_outlier = {}
with PdfPages("raster_overall.pdf") as pdf:
    for neuron in spike_inf['Neuron'].unique():
        filter_outlier[neuron] = []
        neuron_df = spike_inf[spike_inf['Neuron'] == neuron]

        fig, ax = plt.subplots(figsize = (12, 7.5))
        for index, row in trigger_time.iterrows():
            date = row['date']
            image_order = row['order']
            start = row['expand_start'] 
            end = row['expand_end']
            
            filtered_spikes = neuron_df[(neuron_df['date'] == date) & 
                                        (neuron_df['time'] >= start) & 
                                        (neuron_df['time'] <= end)]
            
            if not filtered_spikes.empty:
                ax.plot(filtered_spikes['time'] - start, [index] * len(filtered_spikes), marker='|', mew=1, markersize=3, ls='', color='k')
            else:
                filter_outlier[neuron].append(index)

        ax.set_yticks([])
        #ax.axvspan(1 + (5000 - 1) * 0.5, 1 + (5000 - 1), color='gray', alpha=0.3)

        ax.set_xlabel('time (s)')
        ax.set_ylabel("")
        ax.set_title(f'Raster Plot for {neuron}')

        pdf.savefig(fig)
        plt.close()

In [10]:
with PdfPages("raster_split_image.pdf") as pdf:
    for neuron in spike_inf['Neuron'].unique():
        neuron_df = spike_inf[spike_inf['Neuron'] == neuron]


        fig, axs = plt.subplots(3, 4, figsize=(20, 15), sharex=True, sharey=True)
        fig.subplots_adjust(hspace=0.5, wspace=0.5)

        for image in range(1, 13):  
            ax = axs[(image - 1) // 4, (image - 1) % 4]  
            trigger_time_temp = trigger_time[trigger_time['image'] == image]

            for index, row in trigger_time_temp.iterrows():
                date = row['date']
                image_order = row['order']
                start = row['expand_start'] 
                end = row['expand_end'] 

                filtered_spikes = neuron_df[(neuron_df['date'] == date) & 
                                            (neuron_df['time'] >= start) & 
                                            (neuron_df['time'] <= end)]

                if not filtered_spikes.empty:
                    ax.plot(filtered_spikes['time'] - start, [index] * len(filtered_spikes), 
                            marker='|', mew=1, markersize=3, ls='', color='k')


            ax.set_xticks([])
            ax.set_yticks([])

        pdf.savefig(fig)
        plt.close()

In [11]:
mean_spike_rate_dict = {}

with PdfPages("peth_lineplot_all.pdf") as pdf:
    for neuron in spike_inf['Neuron'].unique():
        plt.figure(figsize=(4, 2.5))
        spike_rates_all = np.empty((99))

        for image in range(1, 13):
            neuron_df = spike_inf[spike_inf['Neuron'] == neuron]

            dates = trigger_time[trigger_time['image'] == image]['date'].unique()
            date_indices = np.arange(len(dates))  

            spike_rates = []
            std_rates = []

            for date in dates:
                trigger_time_temp = trigger_time[(trigger_time['image'] == image) & (trigger_time['date'] == date)]
                all_spikes = []

                for index, row in trigger_time_temp.iterrows():
                    start = row['expand_start'] 
                    end = row['expand_end']

                    filtered_spikes = neuron_df[(neuron_df['date'] == date) &
                                                (neuron_df['time'] >= start) &
                                                (neuron_df['time'] <= end)]

                    if not filtered_spikes.empty:
                        relative_spikes = filtered_spikes['time'] - start
                        all_spikes.extend(relative_spikes)

                if all_spikes:
                    time_bins = np.linspace(0, max(all_spikes), 100)
                    spike_counts, _ = np.histogram(all_spikes, bins=time_bins)
                    spike_rate = spike_counts / (time_bins[1] - time_bins[0])
                    spike_rates.append(spike_rate)
                spike_rates_all = np.vstack((spike_rates_all, spike_rates))

        mean_spike_rate = np.mean(spike_rates, axis=0)
        std_spike_rate = np.std(spike_rates, axis=0)

        plt.plot(time_bins[:-1], mean_spike_rate, color='k', linewidth = 2)
        plt.plot(time_bins[:-1], mean_spike_rate - std_spike_rate, color='lightgray', alpha=0.7, linewidth = 1)
        plt.plot(time_bins[:-1], mean_spike_rate + std_spike_rate, color='lightgray', alpha=0.7, linewidth = 1)
        plt.title(neuron)
        plt.xticks([])
        plt.yticks([])
        pdf.savefig()
        plt.close()

In [12]:
mean_spike_rate_dict = {}

with PdfPages("peth_lineplot.pdf") as pdf:
    for image in range(1, 13):
        fig, axes = plt.subplots(3, 7, figsize=(15, 9))
        axes = axes.flatten()
        num = 0
        image_mean_spike_rate_data = []
        for neuron in spike_inf['Neuron'].unique():
            ax = axes[num]
            neuron_df = spike_inf[spike_inf['Neuron'] == neuron]

            dates = trigger_time[trigger_time['image'] == image]['date'].unique()
            date_indices = np.arange(len(dates))  

            spike_rates = []
            std_rates = []

            for date in dates:
                trigger_time_temp = trigger_time[(trigger_time['image'] == image) & (trigger_time['date'] == date)]
                all_spikes = []

                for index, row in trigger_time_temp.iterrows():
                    start = row['expand_start']
                    end = row['expand_end']

                    filtered_spikes = neuron_df[(neuron_df['date'] == date) &
                                                (neuron_df['time'] >= start) &
                                                (neuron_df['time'] <= end)]

                    if not filtered_spikes.empty:
                        relative_spikes = filtered_spikes['time'] - start
                        all_spikes.extend(relative_spikes)

                if all_spikes:
                    time_bins = np.linspace(0, max(all_spikes), 100)
                    spike_counts, _ = np.histogram(all_spikes, bins=time_bins)
                    spike_rate = spike_counts / (time_bins[1] - time_bins[0])
                    spike_rates.append(spike_rate)

            mean_spike_rate = np.mean(spike_rates, axis=0)
            std_spike_rate = np.std(spike_rates, axis=0)

            image_mean_spike_rate_data.append({
                'Neuron': neuron,
                'Mean_Spike_Rate': mean_spike_rate
            })

            ax.plot(time_bins[:-1], mean_spike_rate, color='k', linewidth = 2)
            ax.plot(time_bins[:-1], mean_spike_rate - std_spike_rate, color='lightgray', alpha=0.7, linewidth = 1)
            ax.plot(time_bins[:-1], mean_spike_rate + std_spike_rate, color='lightgray', alpha=0.7, linewidth = 1)
            #ax.set_ylim(- np.mean(mean_spike_rate) , 2 *np.mean(mean_spike_rate))
            #ax.fill_between(time_bins[:-1], mean_spike_rate - std_spike_rate, mean_spike_rate + std_spike_rate, color='lightgray', alpha=0.5, label='Std Deviation')
            quarter = (time_bins[-1] - time_bins[0]) * 0.25 + time_bins[0]
            three_quarter = (time_bins[-1] - time_bins[0]) * 0.75 + time_bins[0]
            ax.annotate("", xy=(quarter, np.max(mean_spike_rate + std_spike_rate) * 0.9), xytext=(quarter, np.max(mean_spike_rate + std_spike_rate)),
                        arrowprops=dict(arrowstyle="-|>", color="black"))
            ax.annotate("", xy=(three_quarter, np.max(mean_spike_rate + std_spike_rate) * 0.9), xytext=(three_quarter, np.max(mean_spike_rate + std_spike_rate)),
                        arrowprops=dict(arrowstyle="-|>", color="black"))
            ax.set_xlabel('')
            ax.set_ylabel('')
            ax.set_title(f'{neuron}', fontsize=10)
            ax.legend()

            ax.set_xticks([])
            ax.set_yticks([])
            ax.grid(False)
            ax.legend().set_visible(False)
            num += 1
        axes[19].axis('off')
        axes[18].axis('off')
        fig.suptitle(f"Image_{image}", fontsize=20)
        fig.text(0.5, 0.04, 'Time', ha='center', fontsize=14)
        fig.text(0.06, 0.5, 'Spike Rate (Hz)', va='center', rotation='vertical', fontsize=14)
        pdf.savefig(fig)
        plt.close()

        mean_spike_rate_dict[image] = pd.DataFrame(image_mean_spike_rate_data)

In [13]:
spike_rates_all_dict = {}

for neuron in select_neurons:
    spike_rates_all_dict[neuron] = pd.DataFrame()
    for image in range(1, 19):
        neuron_df = spike_inf[spike_inf['Neuron'] == neuron]

        dates = trigger_time[trigger_time['image'] == image]['date'].unique()
        date_indices = np.arange(len(dates))  

        spike_rates = []
        std_rates = []

        for date in dates:
            trigger_time_temp = trigger_time[(trigger_time['image'] == image) & (trigger_time['date'] == date)]
            all_spikes = []

            for index, row in trigger_time_temp.iterrows():
                start = row['expand_start']
                end = row['expand_end']

                filtered_spikes = neuron_df[(neuron_df['date'] == date) &
                                            (neuron_df['time'] >= start) &
                                            (neuron_df['time'] <= end)]

                if not filtered_spikes.empty:
                    relative_spikes = filtered_spikes['time'] - start
                    all_spikes.extend(relative_spikes)

            if all_spikes:
                time_bins = np.linspace(0, max(all_spikes), 500)
                spike_counts, _ = np.histogram(all_spikes, bins=time_bins)
                spike_rate = spike_counts / (time_bins[1] - time_bins[0])
                spike_rates.append(spike_rate)

            spike_rates_all_dict[neuron] = pd.concat((spike_rates_all_dict[neuron], pd.DataFrame(spike_rates)), axis= 0)

In [14]:
split_point = []
with PdfPages("peth_diff.pdf") as pdf:
    for neuron in spike_rates_all_dict.keys():
        max_x = np.argmax(spike_rates_all_dict[neuron].mean(axis = 0)[110:150]) + 110
        split_point.append(max_x)
        plt.figure()
        plt.plot(range(499), spike_rates_all_dict[neuron].mean(axis = 0), alpha=0.7, color = "#70A5D9")
        plt.scatter(max_x, spike_rates_all_dict[neuron].mean(axis = 0)[max_x], color='#F0868C',s=60)
        pdf.savefig()
        plt.close()


In [15]:
with PdfPages("peth_diff_overlay.pdf") as pdf:
    plt.figure(figsize=(10, 6))  
    
    colors = ['#70A5D9', '#F0868C', '#AFC7E8', '#F9BEB9', '#5977A0', '#D8576B', '#3B526E', '#E6919C']  
    color_index = 0  

    split_point = []
    for neuron in spike_rates_all_dict.keys():
        max_x = np.argmax(spike_rates_all_dict[neuron].mean(axis = 0)[110:150]) + 110
        split_point.append(max_x)
        
        plt.plot(range(499), spike_rates_all_dict[neuron].mean(axis=0), alpha=0.7, color=colors[color_index % len(colors)], label=f'Neuron {neuron}')
        plt.scatter(max_x, spike_rates_all_dict[neuron].mean(axis=0)[max_x], color='#F0868C', s=60)
        color_index += 1  
    
    plt.tight_layout()
    
    pdf.savefig()
    plt.close()

In [16]:
from scipy import stats

num_neurons = len(spike_inf['Neuron'].unique())
num_images = 12
result_matrix = np.zeros((num_neurons, num_images), dtype=int)
pvalue_matrix = np.ones((num_neurons, num_images))
diff_matrix = np.ones((num_neurons, num_images))

for image in range(1, 13):
    num = 0
    image_mean_spike_rate_data = []
    for neuron_idx, neuron in enumerate(spike_inf['Neuron'].unique()):

        neuron_df = spike_inf[spike_inf['Neuron'] == neuron]

        dates = trigger_time[trigger_time['image'] == image]['date'].unique()
        date_indices = np.arange(len(dates))  

        spike_rates_first = []
        spike_rates_second = []

        for date in dates:
            trigger_time_temp = trigger_time[(trigger_time['image'] == image) & (trigger_time['date'] == date)]
            all_spikes_first = []
            all_spikes_second = []

            for index, row in trigger_time_temp.iterrows():
                start = row['start'] 
                end = row['expand_start'] + 30000/500 * np.array(split_point).mean()

                filtered_spikes_first = neuron_df[(neuron_df['date'] == date) &
                                                  (neuron_df['time'] >= start) &
                                                  (neuron_df['time'] <= end)]

                if not filtered_spikes_first.empty:
                    relative_spikes_first = filtered_spikes_first['time'] - start
                    all_spikes_first.extend(relative_spikes_first)

                start_2 = row['start'] - 30000/500 * np.array(split_point).mean() + 5000
                end_2 = row['start'] 

                filtered_spikes_second = neuron_df[(neuron_df['date'] == date) &
                                                   (neuron_df['time'] >= start_2) &
                                                   (neuron_df['time'] <= end_2)]

                if not filtered_spikes_second.empty:
                    relative_spikes_second = filtered_spikes_second['time'] - start_2
                    all_spikes_second.extend(relative_spikes_second)

            if all_spikes_first:
                time_bins_first = np.linspace(0, max(all_spikes_first), 100)
                spike_counts_first, _ = np.histogram(all_spikes_first, bins=time_bins_first)
                spike_rate_first = spike_counts_first / (time_bins_first[1] - time_bins_first[0])
                spike_rates_first.append(spike_rate_first)

            if all_spikes_second:
                time_bins_second = np.linspace(0, max(all_spikes_second), 100)
                spike_counts_second, _ = np.histogram(all_spikes_second, bins=time_bins_second)
                spike_rate_second = spike_counts_second / (time_bins_second[1] - time_bins_second[0])
                spike_rates_second.append(spike_rate_second)

        mean_spike_rate_first = np.mean(spike_rates_first, axis=0) if spike_rates_first else np.zeros(100)
        mean_spike_rate_second = np.mean(spike_rates_second, axis=0) if spike_rates_second else np.zeros(100)

        t_stat, p_value = stats.ttest_ind(mean_spike_rate_first, mean_spike_rate_second, nan_policy='omit')

        if p_value < 0.01:
            if np.mean(mean_spike_rate_first) > np.mean(mean_spike_rate_second):
                result_matrix[neuron_idx, image-1] = 1
                diff_matrix[neuron_idx, image-1] = np.mean(mean_spike_rate_first)/np.mean(mean_spike_rate_second)
            else:
                result_matrix[neuron_idx, image-1] = -1
                diff_matrix[neuron_idx, image-1] = np.mean(mean_spike_rate_second)/np.mean(mean_spike_rate_first)
        else:
            result_matrix[neuron_idx, image-1] = 0
            diff_matrix[neuron_idx, image-1] = 1
        pvalue_matrix[neuron_idx, image-1] = p_value
        


neurons = spike_inf['Neuron'].unique()
images = range(1, 13)
result_df_phase1 = pd.DataFrame(result_matrix, index=neurons, columns=images)
pvalue_df_phase1 = pd.DataFrame(pvalue_matrix, index=neurons, columns=images)
diff_df_phase1 = pd.DataFrame(diff_matrix, index=neurons, columns=images)

result_df_phase1.to_csv("glimpse_result_df.csv")
pvalue_df_phase1.to_csv("glimpse_pvalue_df.csv")
diff_df_phase1.to_csv("glimpse_diff_df.csv")

In [17]:
from math import pi

result_df_phase1 = result_df_phase1[[1, 7,2,8,3,9,4,10,5,11,6,12]]
pvalue_df_phase1 = pvalue_df_phase1[[1, 7,2,8,3,9,4,10,5,11,6,12]]

neurons = result_df_phase1.index
images = result_df_phase1.columns

all_pvalues = pvalue_df_phase1.values.flatten()
all_pvalues = all_pvalues[all_pvalues != 0]  
all_log_pvalues = -np.log(all_pvalues)

with PdfPages('glimpse_neuron_radar_plots.pdf') as pdf:
    for neuron in neurons:
        fig = plt.figure(figsize=(5, 4))
        ax = fig.add_subplot(111, polar=True)
        
        results = result_df_phase1.loc[neuron].values
        pvalues = pvalue_df_phase1.loc[neuron].values
        
        angles = np.linspace(0, 2*np.pi, len(images), endpoint=False).tolist()
        
        for j, (result, pvalue, image) in enumerate(zip(results, pvalues, images)):
            if pvalue == 0:
                height = 0
            else:
                height = -np.log(pvalue)
            
            if result == 1:
                color = "#F9BEB9"
            elif result == 0:
                color = 'lightgrey'
            else:
                color = '#AFC7E8'
            
            ax.bar(angles[j], height, width=2*np.pi/len(images), bottom=0.0, color=color, alpha=0.8)
                
        ax.set_xticks(angles)
        ax.set_xticklabels(images)
        ax.set_title(neuron, pad=20, x=-0.1)
        
        ax.grid(True, color='gray', linestyle='--', linewidth=0.5, alpha=0.5)
        ax.tick_params(axis='y', labelsize=8)
        plt.tight_layout()
        
        pdf.savefig()
        plt.close()

In [25]:
from matplotlib.colors import ListedColormap

with PdfPages("glimpse_neuron_reaction.pdf") as pdf:
    cmap = ListedColormap(['#70A5D9', 'lightgrey', '#F0868C'])
    plt.figure(figsize=(4, 4))
    ax = sns.heatmap(result_df_phase1, cmap=cmap, cbar=True)
    pdf.savefig()
    plt.close()

    cmap = ListedColormap(['#70A5D9', 'lightgrey', '#F0868C'])
    plt.figure(figsize=(4, 4))
    ax = sns.heatmap(result_df_phase1, cmap=cmap, cbar=False, linecolor="white",   
    linewidths=0.3)
    ax.xaxis.set_ticks([])
    ax.yaxis.set_ticks([])
    ax.axis('off') 
    pdf.savefig()
    plt.close()

    plt.figure(figsize=(4, 4))
    ax = sns.heatmap(-np.log(pvalue_df_phase1), cmap='coolwarm', cbar=True, annot=True,fmt='.2f',
    annot_kws={"size": 4})
    ax.xaxis.set_ticks([])
    ax.yaxis.set_ticks([])
    ax.axis('off') 
    pdf.savefig()
    plt.close()

    plt.figure(figsize=(4, 4))
    ax = sns.heatmap(-np.log(pvalue_df_phase1), cmap='coolwarm', cbar=False, annot=False,fmt='.2f',
    annot_kws={"size": 4})
    ax.xaxis.set_ticks([])
    ax.yaxis.set_ticks([])
    ax.axis('off') 
    pdf.savefig()
    plt.close()

In [19]:
from scipy import stats

num_neurons = len(spike_inf['Neuron'].unique())
num_images = 12
result_matrix = np.zeros((num_neurons, num_images), dtype=int)
pvalue_matrix = np.ones((num_neurons, num_images))
diff_matrix = np.ones((num_neurons, num_images))

for image in range(1, 13):
    num = 0
    image_mean_spike_rate_data = []
    for neuron_idx, neuron in enumerate(spike_inf['Neuron'].unique()):

        neuron_df = spike_inf[spike_inf['Neuron'] == neuron]

        dates = trigger_time[trigger_time['image'] == image]['date'].unique()
        date_indices = np.arange(len(dates))  

        spike_rates_first = []
        spike_rates_second = []

        for date in dates:
            trigger_time_temp = trigger_time[(trigger_time['image'] == image) & (trigger_time['date'] == date)]
            all_spikes_first = []
            all_spikes_second = []

            for index, row in trigger_time_temp.iterrows():
                start = row['start'] + 10000
                end = row['end']

                filtered_spikes_first = neuron_df[(neuron_df['date'] == date) &
                                                  (neuron_df['time'] >= start) &
                                                  (neuron_df['time'] <= end)]

                if not filtered_spikes_first.empty:
                    relative_spikes_first = filtered_spikes_first['time'] - start
                    all_spikes_first.extend(relative_spikes_first)

                start_2 = row['start']- (end - start)
                end_2 = row['start'] 

                filtered_spikes_second = neuron_df[(neuron_df['date'] == date) &
                                                   (neuron_df['time'] >= start_2) &
                                                   (neuron_df['time'] <= end_2)]

                if not filtered_spikes_second.empty:
                    relative_spikes_second = filtered_spikes_second['time'] - start_2
                    all_spikes_second.extend(relative_spikes_second)

            if all_spikes_first:
                time_bins_first = np.linspace(0, max(all_spikes_first), 500)
                spike_counts_first, _ = np.histogram(all_spikes_first, bins=time_bins_first)
                spike_rate_first = spike_counts_first / (time_bins_first[1] - time_bins_first[0])
                spike_rates_first.append(spike_rate_first)

            if all_spikes_second:
                time_bins_second = np.linspace(0, max(all_spikes_second), 500)
                spike_counts_second, _ = np.histogram(all_spikes_second, bins=time_bins_second)
                spike_rate_second = spike_counts_second / (time_bins_second[1] - time_bins_second[0])
                spike_rates_second.append(spike_rate_second)

        mean_spike_rate_first = np.mean(spike_rates_first, axis=0) if spike_rates_first else np.zeros(100)
        mean_spike_rate_second = np.mean(spike_rates_second, axis=0) if spike_rates_second else np.zeros(100)

        t_stat, p_value = stats.ttest_ind(mean_spike_rate_first, mean_spike_rate_second, nan_policy='omit')

        if p_value < 1e-3:
            if np.mean(mean_spike_rate_first) > np.mean(mean_spike_rate_second):
                result_matrix[neuron_idx, image-1] = 1
            else:
                result_matrix[neuron_idx, image-1] = -1
        else:
            result_matrix[neuron_idx, image-1] = 0
        pvalue_matrix[neuron_idx, image-1] = p_value
        diff_matrix[neuron_idx, image-1] = np.mean(mean_spike_rate_first)/np.mean(mean_spike_rate_second)


neurons = spike_inf['Neuron'].unique()
images = range(1, 13)
result_df_phase2 = pd.DataFrame(result_matrix, index=neurons, columns=images)
pvalue_df_phase2 = pd.DataFrame(pvalue_matrix, index=neurons, columns=images)
diff_df_phase2 = pd.DataFrame(diff_matrix, index=neurons, columns=images)

result_df_phase2.to_csv("last_result_df.csv")
pvalue_df_phase2.to_csv("last_pvalue_df.csv")
diff_df_phase2.to_csv("last_diff_df.csv")

In [21]:
result_df_phase2 = result_df_phase2[[1, 7,2,8,3,9,4,10,5,11,6,12]]
pvalue_df_phase2 = pvalue_df_phase2[[1, 7,2,8,3,9,4,10,5,11,6,12]]

neurons = result_df_phase2.index
images = result_df_phase2.columns


with PdfPages('last_neuron_radar_plots.pdf') as pdf:
    for neuron in neurons:
        fig = plt.figure(figsize=(5, 4))
        ax = fig.add_subplot(111, polar=True)
        
        results = result_df_phase2.loc[neuron].values
        pvalue = pvalue_df_phase2.loc[neuron].values
        pvalue = -np.log(pvalue)
        
        angles = np.linspace(0, 2*np.pi, len(images), endpoint=False).tolist()
        
        for j, (result, pvalue, image) in enumerate(zip(results, pvalue, images)):
            if pvalue == 0:
                height = 0
            else:
                height = pvalue
            
            if result == 1:
                color = "#F9BEB9"
            elif result == 0:
                color = 'lightgrey'
            else:
                color = '#AFC7E8'
            
            ax.bar(angles[j], height, width=2*np.pi/len(images), bottom=0.0, color=color)
        
        
        ax.set_xticks(angles)
        ax.set_xticklabels(images)
        ax.set_title(neuron, pad=20, x=-0.1)
        
        ax.grid(True, color='gray', linestyle='--', linewidth=0.5, alpha=0.5)
        
        plt.tight_layout()
        
        pdf.savefig()
        plt.close()

In [23]:
with PdfPages("last_neuron_reaction.pdf") as pdf:
    cmap = ListedColormap(['#70A5D9', 'lightgrey', '#F0868C'])
    plt.figure(figsize=(4, 4))
    ax = sns.heatmap(result_df_phase2, cmap=cmap, cbar=True)
    pdf.savefig()
    plt.close()

    cmap = ListedColormap(['#70A5D9', 'lightgrey', '#F0868C'])
    plt.figure(figsize=(4, 4))
    ax = sns.heatmap(result_df_phase2, cmap=cmap, cbar=False, linecolor="white",   
    linewidths=0.3)
    ax.xaxis.set_ticks([])
    ax.yaxis.set_ticks([])
    ax.axis('off') 
    pdf.savefig()
    plt.close()

    plt.figure(figsize=(4, 4))
    ax = sns.heatmap(-np.log(pvalue_df_phase2), cmap='coolwarm', cbar=True, annot=True,fmt='.2f',
    annot_kws={"size": 4})
    ax.xaxis.set_ticks([])
    ax.yaxis.set_ticks([])
    ax.axis('off') 
    pdf.savefig()
    plt.close()

    plt.figure(figsize=(4, 4))
    ax = sns.heatmap(-np.log(pvalue_df_phase2), cmap='coolwarm', cbar=False, annot=False,fmt='.2f',
    annot_kws={"size": 4}, vmax=100)
    ax.xaxis.set_ticks([])
    ax.yaxis.set_ticks([])
    ax.axis('off') 
    pdf.savefig()
    plt.close()

In [26]:
sum_spike_rate_dict = {}
window_starts = np.arange(0, 20000-500+1, 50) 
n_windows = len(window_starts)  

for image in range(1, 13):
    image_mean_spike_rate_data = []
    
    for neuron in spike_inf['Neuron'].unique():
        neuron_df = spike_inf[spike_inf['Neuron'] == neuron]
        dates = trigger_time[trigger_time['image'] == image]['date'].unique()
        
        all_rates = []
        
        for date in dates:
            trigger_time_temp = trigger_time[(trigger_time['image'] == image) 
                                          & (trigger_time['date'] == date)]
            window_counts = np.zeros(n_windows)
            
            for _, row in trigger_time_temp.iterrows():
                start = row['start']
                end = row['end']
                
                filtered_spikes = neuron_df[(neuron_df['date'] == date) 
                                          & (neuron_df['time'] >= start)
                                          & (neuron_df['time'] <= end)]
                
                if not filtered_spikes.empty:
                    relative_spikes = filtered_spikes['time'].values - start
                    
                    spikes_matrix = relative_spikes[:, np.newaxis]  
                    in_window = (spikes_matrix >= window_starts) \
                              & (spikes_matrix < (window_starts + 100))
                    window_counts += np.sum(in_window, axis=0)

            spike_rate = window_counts
            all_rates.append(spike_rate)
        
        mean_spike_rate = np.mean(all_rates, axis=0) if all_rates else np.zeros(n_windows)
        
        image_mean_spike_rate_data.append({
            'Neuron': neuron,
            'Mean_Spike_Rate': mean_spike_rate
        })
    
    sum_spike_rate_dict[image] = pd.DataFrame(image_mean_spike_rate_data)

In [27]:
for image in sum_spike_rate_dict.keys():
    temp = pd.DataFrame()
    for i in range(len(sum_spike_rate_dict[1])):
        temp = pd.concat((temp, pd.DataFrame(sum_spike_rate_dict[image].iloc[i, 1])), axis=1)
    temp = temp.T
    temp.index = sum_spike_rate_dict[1]['Neuron']
    temp.to_csv(f"sum_spike_rate/Image_{image}_sum_spike_rate.csv")

In [28]:
(30000/500 * np.array(split_point).mean() - 5000)/ 20000 * 391

60.95690000000001

In [29]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

def latent_variables_pc(filepath):
    pca = PCA(n_components=10)
    data = pd.read_csv(filepath,index_col=0)
    data = data.iloc[:, :58]
    data = data.T
    scaler = StandardScaler()
    data_normalized = scaler.fit_transform(data)
    #data_normalized = data_normalized.T
    data_pca = pca.fit_transform(data_normalized)
    return data_pca, pca.explained_variance_ratio_

In [30]:
data_pca_dict = {}
pc_ratio_dict = {}
spike_rate_dir = ['Image_1_sum_spike_rate.csv', 'Image_2_sum_spike_rate.csv', 'Image_3_sum_spike_rate.csv', 'Image_4_sum_spike_rate.csv', 'Image_5_sum_spike_rate.csv', 'Image_6_sum_spike_rate.csv',
                  'Image_7_sum_spike_rate.csv', 'Image_8_sum_spike_rate.csv', 'Image_9_sum_spike_rate.csv', 'Image_10_sum_spike_rate.csv', 'Image_11_sum_spike_rate.csv', 'Image_12_sum_spike_rate.csv']

for file in spike_rate_dir:
    data_pca_dict[file], pc_ratio_dict[file] = latent_variables_pc(f'/media/ubuntu/sda/data/mouse11/output/analysis_grating/03_single_neuron/dynamic/sum_spike_rate/{file}')

In [31]:
data_pc1 = pd.DataFrame()
data_pc2 = pd.DataFrame()
data_pc3 = pd.DataFrame()

for file in spike_rate_dir:
    data_pc1 = pd.concat((data_pc1, pd.DataFrame(data_pca_dict[file][:, 0])), axis=1)
    data_pc2 = pd.concat((data_pc2, pd.DataFrame(data_pca_dict[file][:, 1])), axis=1)
    data_pc3 = pd.concat((data_pc3, pd.DataFrame(data_pca_dict[file][:, 2])), axis=1)

In [32]:
os.makedirs("processed_data", exist_ok=True)
data_pc1.to_csv("processed_data/data_pc1_phase1.csv")
data_pc2.to_csv("processed_data/data_pc2_phase1.csv")
data_pc3.to_csv("processed_data/data_pc3_phase1.csv")

In [33]:
import torch.nn.functional as F
import torch
os.makedirs("neuron_model", exist_ok=True)
with PdfPages("neuron_model/similarity_heatmap_phase1.pdf") as pdf:

    data_pc1.columns = [1,2,3,4,5,6,7,8,9,10,11,12]
    #data_pc1 = data_pc1[[1, 5,6,7,10,11,12,14,17,18, 2,3,4,8,9,13,15,16]]
    data = torch.tensor(data_pc1.T.to_numpy())
    data.unsqueeze_(1) 
    similarity_matrix = F.cosine_similarity(data, data.transpose(0, 1), dim=2)
    sns.heatmap(similarity_matrix.numpy(), cmap="coolwarm", vmin=0, vmax=1)
    pdf.savefig()
    plt.close()

    plt.figure(figsize=(5,5))
    sns.heatmap(similarity_matrix.numpy(), cmap="coolwarm", vmin=0, vmax=1, cbar=False, xticklabels=False, yticklabels=False)
    pdf.savefig()
    plt.close()

    data_pc2.columns = [1,2,3,4,5,6,7,8,9,10,11,12]
    #data_pc2 = data_pc2[[1,2,3,4,5,6, 9,10,12, 13, 14, 15, 16, 17, 7,8,11,18]]
    data = torch.tensor(data_pc2.T.to_numpy())
    data.unsqueeze_(1) 
    similarity_matrix_2 = F.cosine_similarity(data, data.transpose(0, 1), dim=2)
    sns.heatmap(similarity_matrix_2.numpy(), cmap="coolwarm", vmin=0, vmax=1)
    pdf.savefig()
    plt.close()

    plt.figure(figsize=(5,5))
    sns.heatmap(similarity_matrix_2.numpy(), cmap="coolwarm", vmin=0, vmax=1, cbar=False, xticklabels=False, yticklabels=False)
    pdf.savefig()
    plt.close()

    data_pc3.columns = [1,2,3,4,5,6,7,8,9,10,11,12]
    data_pc3 = data_pc3[[1,7, 12, 2,3,4,5,6,8,9,10,11]]
    data = torch.tensor(data_pc3.T.to_numpy())
    data.unsqueeze_(1) 
    similarity_matrix_3 = F.cosine_similarity(data, data.transpose(0, 1), dim=2)
    sns.heatmap(similarity_matrix_3.numpy(), cmap="coolwarm", vmin=0, vmax=1)
    pdf.savefig()
    plt.close()

    plt.figure(figsize=(5,5))
    sns.heatmap(similarity_matrix_3.numpy(), cmap="coolwarm", vmin=0, vmax=1, cbar=False, xticklabels=False, yticklabels=False)
    pdf.savefig()
    plt.close()

In [34]:
with PdfPages("neuron_model/waveform_phase1.pdf") as pdf:
    plt.figure(figsize=(4, 3))
    for file in spike_rate_dir:
        sns.lineplot(data_pca_dict[file][:, 0])
    pdf.savefig()
    plt.close()

    plt.figure(figsize=(4, 3))
    for file in spike_rate_dir:
        sns.lineplot(data_pca_dict[file][:, 1])
    pdf.savefig()
    plt.close()


    plt.figure(figsize=(4, 3))
    for file in spike_rate_dir:
        sns.lineplot(data_pca_dict[file][:, 2])
    pdf.savefig()
    plt.close()

In [35]:
pc_ratio = pd.DataFrame()

for key ,item in pc_ratio_dict.items():
    pc_ratio = pd.concat((pc_ratio, pd.DataFrame(item)), axis=1)

pc_ratio = pc_ratio.mean(axis=1)
cumulative_sums = []
current_sum = 0
for value in pc_ratio:
    current_sum += value
    cumulative_sums.append(current_sum)

In [36]:
pd.DataFrame(cumulative_sums).to_csv("processed_data/pc_ratio_phase1.csv")

In [37]:
with PdfPages('neuron_model/pc_ration_phase1.pdf') as pdf:
    plt.figure(figsize=(3,2))
    sns.lineplot(x = range(10),
                 y = pc_ratio,
                 color = 'black')
    plt.ylim(0, 1)
    pdf.savefig()
    plt.close()

In [38]:
import plotly.graph_objects as go
import numpy as np

fig = go.Figure()

colors = ['rgb(31, 119, 180)', 'rgb(255, 127, 14)', 'rgb(44, 160, 44)', 'rgb(214, 39, 40)', 
          'rgb(148, 103, 189)', 'rgb(140, 86, 75)', 'rgb(227, 119, 194)', 'rgb(127, 127, 127)', 
          'rgb(188, 189, 34)', 'rgb(23, 190, 207)']

for i, (key, matrix) in enumerate(data_pca_dict.items()):
    color = colors[i % len(colors)]
    
    x = matrix[:, 0]
    y = matrix[:, 1]
    z = matrix[:, 2]
    
    fig.add_trace(go.Scatter3d(
        x=x,
        y=y,
        z=z,
        mode='lines',
        line=dict(color=color, width=4),
        name=key
    ))

fig.update_layout(
    title='3D Line Plots from Dictionary',
    scene=dict(
        xaxis_title='X Axis',
        yaxis_title='Y Axis',
        zaxis_title='Z Axis',
        xaxis=dict(showgrid=False, gridcolor='white', backgroundcolor='white'),
        yaxis=dict(showgrid=False, gridcolor='white', backgroundcolor='white'),
        zaxis=dict(showgrid=False, gridcolor='white', backgroundcolor='white'),
        bgcolor='white' 
    )
)


fig.show()

- phase 2

In [39]:
def latent_variables_pc(filepath):
    from sklearn.preprocessing import StandardScaler
    from sklearn.decomposition import PCA
    pca = PCA(n_components=10)
    data = pd.read_csv(filepath,index_col=0)
    data = data.iloc[:, 58:]
    data = data.T
    scaler = StandardScaler()
    data_normalized = scaler.fit_transform(data)
    #data_normalized = data_normalized.T
    data_pca = pca.fit_transform(data_normalized)
    return data_pca, pca.explained_variance_ratio_

In [40]:
data_pca_dict = {}
pc_ratio_dict = {}
spike_rate_dir = ['Image_1_sum_spike_rate.csv', 'Image_2_sum_spike_rate.csv', 'Image_3_sum_spike_rate.csv', 'Image_4_sum_spike_rate.csv', 'Image_5_sum_spike_rate.csv', 'Image_6_sum_spike_rate.csv',
                  'Image_7_sum_spike_rate.csv', 'Image_8_sum_spike_rate.csv', 'Image_9_sum_spike_rate.csv', 'Image_10_sum_spike_rate.csv', 'Image_11_sum_spike_rate.csv', 'Image_12_sum_spike_rate.csv']

for file in spike_rate_dir:
    data_pca_dict[file], pc_ratio_dict[file] = latent_variables_pc(f'/media/ubuntu/sda/data/mouse11/output/analysis_grating/03_single_neuron/dynamic/sum_spike_rate/{file}')

In [41]:
data_pc1 = pd.DataFrame()
data_pc2 = pd.DataFrame()
data_pc3 = pd.DataFrame()

for file in spike_rate_dir:
    data_pc1 = pd.concat((data_pc1, pd.DataFrame(data_pca_dict[file][:, 0])), axis=1)
    data_pc2 = pd.concat((data_pc2, pd.DataFrame(data_pca_dict[file][:, 1])), axis=1)
    data_pc3 = pd.concat((data_pc3, pd.DataFrame(data_pca_dict[file][:, 2])), axis=1)

In [42]:
data_pc1.shape

(333, 12)

In [43]:
data_pc1.to_csv("processed_data/data_pc1_phase2.csv")
data_pc2.to_csv("processed_data/data_pc2_phase2.csv")
data_pc3.to_csv("processed_data/data_pc3_phase2.csv")

In [44]:
with PdfPages("neuron_model/similarity_heatmap_phase2.pdf") as pdf:

    data_pc1.columns = [1,2,3,4,5,6,7,8,9,10,11,12]
    data_pc1 = data_pc1[[1, 4, 2, 3, 5,6,7,8,9,10,11,12]]
    data = torch.tensor(data_pc1.T.to_numpy())
    data.unsqueeze_(1) 
    similarity_matrix = F.cosine_similarity(data, data.transpose(0, 1), dim=2)
    sns.heatmap(similarity_matrix.numpy(), cmap="coolwarm", vmin=0, vmax=1)
    pdf.savefig()
    plt.close()

    plt.figure(figsize=(5,5))
    sns.heatmap(similarity_matrix.numpy(), cmap="coolwarm", vmin=0, vmax=1, cbar=False, xticklabels=False, yticklabels=False)
    pdf.savefig()
    plt.close()

    data_pc2.columns = [1,2,3,4,5,6,7,8,9,10,11,12]
    #data_pc2 = data_pc2[[1,2,3,4,5,6, 9,10,12, 13, 14, 15, 16, 17, 7,8,11,18]]
    data = torch.tensor(data_pc2.T.to_numpy())
    data.unsqueeze_(1) 
    similarity_matrix_2 = F.cosine_similarity(data, data.transpose(0, 1), dim=2)
    sns.heatmap(similarity_matrix_2.numpy(), cmap="coolwarm", vmin=0, vmax=1)
    pdf.savefig()
    plt.close()

    plt.figure(figsize=(5,5))
    sns.heatmap(similarity_matrix_2.numpy(), cmap="coolwarm", vmin=0, vmax=1, cbar=False, xticklabels=False, yticklabels=False)
    pdf.savefig()
    plt.close()

    data_pc3.columns = [1,2,3,4,5,6,7,8,9,10,11,12]
    data = torch.tensor(data_pc3.T.to_numpy())
    data.unsqueeze_(1) 
    similarity_matrix_3 = F.cosine_similarity(data, data.transpose(0, 1), dim=2)
    sns.heatmap(similarity_matrix_3.numpy(), cmap="coolwarm", vmin=0, vmax=1)
    pdf.savefig()
    plt.close()

    plt.figure(figsize=(5,5))
    sns.heatmap(similarity_matrix_3.numpy(), cmap="coolwarm", vmin=0, vmax=1, cbar=False, xticklabels=False, yticklabels=False)
    pdf.savefig()
    plt.close()

In [45]:
with PdfPages("neuron_model/waveform_phase2.pdf") as pdf:
    plt.figure(figsize=(6, 3))
    for file in spike_rate_dir:
        sns.lineplot(data_pca_dict[file][:, 0])
    pdf.savefig()
    plt.close()

    plt.figure(figsize=(6, 3))
    for file in spike_rate_dir:
        sns.lineplot(data_pca_dict[file][:, 1])
    pdf.savefig()
    plt.close()


    plt.figure(figsize=(6, 3))
    for file in spike_rate_dir:
        sns.lineplot(data_pca_dict[file][:, 2])
    pdf.savefig()
    plt.close()

In [46]:
pc_ratio = pd.DataFrame()

for key ,item in pc_ratio_dict.items():
    pc_ratio = pd.concat((pc_ratio, pd.DataFrame(item)), axis=1)

pc_ratio = pc_ratio.mean(axis=1)
cumulative_sums = []
current_sum = 0
for value in pc_ratio:
    current_sum += value
    cumulative_sums.append(current_sum)

In [47]:
pd.DataFrame(cumulative_sums).to_csv("processed_data/pc_ratio_phase2.csv")

In [48]:
with PdfPages('neuron_model/pc_ration_phase2.pdf') as pdf:
    plt.figure(figsize=(3,2))
    sns.lineplot(x = range(10),
                 y = cumulative_sums,
                 color = 'black')
    
    plt.ylim(0, 1)
    pdf.savefig()
    plt.close()

In [49]:
import plotly.graph_objects as go
import numpy as np

fig = go.Figure()

colors = ['rgb(31, 119, 180)', 'rgb(255, 127, 14)', 'rgb(44, 160, 44)', 'rgb(214, 39, 40)', 
          'rgb(148, 103, 189)', 'rgb(140, 86, 75)', 'rgb(227, 119, 194)', 'rgb(127, 127, 127)', 
          'rgb(188, 189, 34)', 'rgb(23, 190, 207)']

for i, (key, matrix) in enumerate(data_pca_dict.items()):
    color = colors[i % len(colors)]
    
    x = matrix[:, 0]
    y = matrix[:, 1]
    z = matrix[:, 2]
    
    fig.add_trace(go.Scatter3d(
        x=x,
        y=y,
        z=z,
        mode='lines',
        line=dict(color=color, width=4),
        name=key
    ))

fig.update_layout(
    title='3D Line Plots from Dictionary',
    scene=dict(
        xaxis_title='X Axis',
        yaxis_title='Y Axis',
        zaxis_title='Z Axis',
        xaxis=dict(showgrid=False, gridcolor='white', backgroundcolor='white'),
        yaxis=dict(showgrid=False, gridcolor='white', backgroundcolor='white'),
        zaxis=dict(showgrid=False, gridcolor='white', backgroundcolor='white'),
        bgcolor='white' 
    )
)


fig.show()

In [50]:
pc_ratio_1 = pd.read_csv("/media/ubuntu/sda/data/mouse11/output/analysis_grating/03_single_neuron/dynamic/processed_data/pc_ratio_phase1.csv", index_col=0)
pc_ratio_2 = pd.read_csv("/media/ubuntu/sda/data/mouse11/output/analysis_grating/03_single_neuron/dynamic/processed_data/pc_ratio_phase2.csv", index_col=0)

In [51]:
with PdfPages('neuron_model/pc_ration_phase12.pdf') as pdf:
    plt.figure(figsize=(3,2))
    sns.lineplot(x = range(10),
                 y = pc_ratio_1['0'],
                 color = '#F0868C',
                 linewidth = 3)
    sns.lineplot(x = range(10),
                 y = pc_ratio_2['0'],
                 color = '#70A5D9',
                 linewidth = 3)
    plt.yticks([])
    plt.xticks([])
    plt.ylim(0, 1.1)
    plt.ylabel("")
    pdf.savefig()
    plt.close()